In [ ]:
!python3 --version

In [ ]:
import pandas

from utils import calculate_percent_replicating_Target,calculate_percent_matching_Target
from utils import plot_simple_comparison, plot_two_comparisons
from utils import enforce_modality_match_order, enforce_timepoint_order, enforce_timepoint_order_in_plot, enforce_modality_match_order_in_plot

In [ ]:
variable = 'dummy'
variable_to_save = variable.replace(' ','_')

experiment_dict = {
    'Cell Line':{
        'Modality':{
            'Density/Timepoint':[]}
            }
    }



In [ ]:
plate_df = pandas.DataFrame()

plate_df = plate_df.append({'Plate Name List' : ['X','Y','Z'], 
variable : 'No', 'Batch' : 'A', 'Percent Replicating' : calculate_percent_replicating_Target(
    'Path_to_A',['X','Y','Z'])}, ignore_index=True)

In [ ]:
plot_simple_comparison(df=plate_df,x=variable,y='Percent Replicating',hue='Batch')

In [ ]:
plate_df.to_csv(f'checkpoint_csvs/{variable_to_save}-replicating.csv',index=False)

In [ ]:
plate_match_df = pandas.DataFrame()

comparison_list = []

for eachline1 in list(experiment_dict.keys()):
    for eachline2 in list(experiment_dict.keys()):
        for eachmodality1 in list(experiment_dict[eachline1].keys()):
            for eachmodality2 in list(experiment_dict[eachline2].keys()):
                for eachtimepoint1 in list(experiment_dict[eachline1][eachmodality1].keys()):
                    for eachtimepoint2 in list(experiment_dict[eachline2][eachmodality2].keys()):
                        string1 = f"{eachline1}-{eachmodality1}-{eachtimepoint1}"
                        string2 = f"{eachline2}-{eachmodality2}-{eachtimepoint2}"
                        if string1 != string2:
                            if f"{string1}-{string2}" not in comparison_list:
                                if f"{string2}-{string1}" not in comparison_list:
                                    comparison_list.append(f"{string1}-{string2}")
                                    plate_match_df = plate_match_df.append({'Percent Matching' : calculate_percent_matching_Target(
                                        '../profiles-cp-jump1/profiles/2020_11_04_CPJUMP1',
                                        experiment_dict[eachline1][eachmodality1][eachtimepoint1],eachmodality1, 
                                        '../profiles-cp-jump1/profiles/2020_11_04_CPJUMP1',
                                        experiment_dict[eachline2][eachmodality2][eachtimepoint2],eachmodality2),
                                        'Modality Match' : enforce_modality_match_order(eachmodality1,eachmodality2), 
                                        'Treatment timepoint':enforce_timepoint_order(eachtimepoint1,eachtimepoint2),
                                        variable:f"{eachline1} - {eachline2}"}, ignore_index=True)

In [ ]:
# Adjust below as needed
hue_order = enforce_timepoint_order_in_plot(list(plate_match_df['Treatment timepoint']))
col_order = enforce_modality_match_order_in_plot(list(plate_match_df['Modality Match']))

x = variable
y = 'Percent Matching'
hue = 'Treatment timepoint'
col = 'Modality Match'


In [ ]:
plot_simple_comparison(df=plate_match_df,x=x,y=y,hue=hue,
hue_order=hue_order, col = col, col_order = col_order, col_wrap=3,ylim=[0.05,0.4],
plotname=f"{x}-{y}-{hue}-{col}.png")

In [ ]:
plate_match_df.to_csv(f'checkpoint_csvs/{variable_to_save}-matching.csv',index=False)